In [16]:
from pyspark.sql.functions import countDistinct

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('testing').getOrCreate()

In [28]:
data = spark.read.csv(r"C:\Users\Asus\Desktop\winequality-red.csv", inferSchema=True, header = True)

In [31]:
data.columns

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [32]:
from pyspark.ml.feature import VectorAssembler

In [33]:
assembler = VectorAssembler(inputCols=['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol'], outputCol='features')

In [34]:
model = assembler.transform(data)

In [42]:
final = model.select('features', 'quality' )

In [43]:
from pyspark.ml.classification import LogisticRegression

In [44]:
lr = LogisticRegression(labelCol='quality', featuresCol='features')

In [45]:
train_data, test_data = final.randomSplit([0.7, 0.3])

In [46]:
final_out = lr.fit(train_data)

In [60]:
predictions = final_out.transform(test_data)

In [55]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [67]:
evaluator = BinaryClassificationEvaluator(labelCol='quality', rawPredictionCol='prediction')

In [73]:
predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|quality|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[4.6,0.52,0.15,2....|      4|[-4.4606980514641...|[7.21900601801146...|       6.0|
|[5.0,0.4,0.5,4.3,...|      6|[-4.0336565559104...|[1.41998672714449...|       7.0|
|[5.1,0.47,0.02,1....|      6|[-4.5060999973238...|[1.49809482088712...|       6.0|
|[5.1,0.51,0.18,2....|      7|[-3.8541469004784...|[1.62799852842516...|       6.0|
|[5.1,0.585,0.0,1....|      7|[-4.0918532179077...|[2.79123133233557...|       6.0|
|[5.2,0.32,0.25,1....|      5|[-3.5837421868649...|[2.79947269778617...|       5.0|
|[5.2,0.48,0.04,1....|      7|[-3.7341241767987...|[4.38334197019366...|       6.0|
|[5.3,0.47,0.11,2....|      7|[-4.0857339939047...|[1.56662783754224...|       7.0|
|[5.3,0.57,0.01,1....|      7|[-4.5670264972857...|[5.19434760877908...|    

In [69]:
predictions.select('quality', 'prediction').show()

+-------+----------+
|quality|prediction|
+-------+----------+
|      4|       6.0|
|      6|       7.0|
|      6|       6.0|
|      7|       6.0|
|      7|       6.0|
|      5|       5.0|
|      7|       6.0|
|      7|       7.0|
|      7|       6.0|
|      5|       5.0|
|      7|       6.0|
|      8|       7.0|
|      5|       5.0|
|      5|       6.0|
|      5|       5.0|
|      5|       6.0|
|      5|       5.0|
|      5|       5.0|
|      6|       6.0|
|      5|       5.0|
+-------+----------+
only showing top 20 rows



In [75]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

evaluatorMulti = MulticlassClassificationEvaluator(labelCol="quality", predictionCol="prediction")
precision = evaluatorMulti.evaluate(predictions, {evaluatorMulti.metricName: "precisionByLabel"})
recall = evaluatorMulti.evaluate(predictions, {evaluatorMulti.metricName: "recallByLabel"})
f1 = evaluatorMulti.evaluate(predictions, {evaluatorMulti.metricName: "f1"})

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Py4JJavaError: An error occurred while calling o704.evaluate.
: java.util.NoSuchElementException: key not found: 0.0
	at scala.collection.MapLike.default(MapLike.scala:236)
	at scala.collection.MapLike.default$(MapLike.scala:235)
	at scala.collection.AbstractMap.default(Map.scala:65)
	at scala.collection.MapLike.apply(MapLike.scala:144)
	at scala.collection.MapLike.apply$(MapLike.scala:143)
	at scala.collection.AbstractMap.apply(Map.scala:65)
	at org.apache.spark.mllib.evaluation.MulticlassMetrics.precision(MulticlassMetrics.scala:150)
	at org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator.evaluate(MulticlassClassificationEvaluator.scala:161)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(Unknown Source)
	at java.lang.reflect.Method.invoke(Unknown Source)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j